In [1]:
import warnings
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_excel('data/appointment.xlsx')
df = df['No of Appointments']
df.head()

0    35
1    21
2    29
3    28
4    31
Name: No of Appointments, dtype: int64

In [3]:
def create_time_series(
                      df, 
                      selective_windows = 30,
                      prediction_windows = 7
                      ):
    values = df.values.squeeze()
    X, Y = [], []
    for i in range(len(values)):
        end_ix = i + selective_windows
        out_end_ix = end_ix + prediction_windows
        if out_end_ix > len(values):
            break
        seq_x, seq_y = values[i:end_ix], values[end_ix:out_end_ix]
        X.append(seq_x)
        Y.append(seq_y)

    return np.array(X), np.array(Y)

In [4]:
X, Y = create_time_series(df)
X = X.reshape((X.shape[0], X.shape[1], 1))
X.shape, Y.shape

((164, 30, 1), (164, 7))

In [5]:
def gru_model():
    """
    Create GRU model
    """
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.GRU(100, activation='relu', input_shape=(X.shape[1], X.shape[2])))
    model.add(tf.keras.layers.Dense(7))
    model.compile(optimizer='adam', loss='mse')
    return model

model = gru_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 100)               30900     
                                                                 
 dense (Dense)               (None, 7)                 707       
                                                                 
Total params: 31,607
Trainable params: 31,607
Non-trainable params: 0
_________________________________________________________________


In [6]:
model.fit(
        X, Y, 
        epochs=1000, 
        verbose=1
        )

Epoch 1/1000
6/6 [==============================] - 9s 50ms/step - loss: 1557.5636
Epoch 2/1000
6/6 [==============================] - 0s 37ms/step - loss: 1402.7107
Epoch 3/1000
6/6 [==============================] - 0s 38ms/step - loss: 1236.3593
Epoch 4/1000
6/6 [==============================] - 0s 38ms/step - loss: 1013.6448
Epoch 5/1000
6/6 [==============================] - 0s 36ms/step - loss: 649.0814
Epoch 6/1000
6/6 [==============================] - 0s 40ms/step - loss: 255.4603
Epoch 7/1000
6/6 [==============================] - 0s 34ms/step - loss: 170.2393
Epoch 8/1000
6/6 [==============================] - 0s 35ms/step - loss: 143.0644
Epoch 9/1000
6/6 [==============================] - 0s 34ms/step - loss: 139.7758
Epoch 10/1000
6/6 [==============================] - 0s 35ms/step - loss: 131.9976
Epoch 11/1000
6/6 [==============================] - 0s 34ms/step - loss: 130.7661
Epoch 12/1000
6/6 [==============================] - 0s 37ms/step - loss: 133.1499
Epoch 13/

In [7]:
model.save('weights/appointment/model.h5')